In [4]:
! pip install numpy
! pip install pandas
! pip install scikit-learn
! pip install matplotlib
! pip install nltk


[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/a6/b6/a9405484fb40746fdc6ae4502b16a9d6e53282ba5baaf9ebe2da579f68c4/matplotlib-3.10.1-cp312-cp312-win_amd64.whl.metadata
  Using cached matplotlib-3.10.1-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/01/c8/fadd0b92ffa7b5eb5949bf340a63a4a496a6930a6c37a7ba0f12acb076d6/contourpy-1.3.2-cp312-cp312-win_amd64.whl.metadata
  Using cached contourpy-1.3.2-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Obtaining dependency information for cycler>=0.10 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629efb316b96de511b418c53d245aae6/cycler-0.12.1-py3-none-any.whl.metadata
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/06/5d/1be5424bb305880e1113631f49a55ea7c7da3a5


[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk

In [78]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to C:\Users\user/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\user/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\user/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\user/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\user/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nlt

True

In [79]:
dataset = pd.read_csv('data/sentimentdataset.csv')

In [80]:
dataset

,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,1,1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,2,2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,3,3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,4,4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,728,732,Collaborating on a science project that receiv...,Happy,2017-08-18 18:20:00,ScienceProjectSuccessHighSchool,Facebook,#ScienceFairWinner #HighSchoolScience,20.0,39.0,UK,2017,8,18,18
728,729,733,Attending a surprise birthday party organized ...,Happy,2018-06-22 14:15:00,BirthdayPartyJoyHighSchool,Instagram,#SurpriseCelebration #HighSchoolFriendship,25.0,48.0,USA,2018,6,22,14
729,730,734,Successfully fundraising for a school charity ...,Happy,2019-04-05 17:30:00,CharityFundraisingTriumphHighSchool,Twitter,#CommunityGiving #HighSchoolPhilanthropy,22.0,42.0,Canada,2019,4,5,17
730,731,735,"Participating in a multicultural festival, cel...",Happy,2020-02-29 20:45:00,MulticulturalFestivalJoyHighSchool,Facebook,#CulturalCelebration #HighSchoolUnity,21.0,43.0,UK,2020,2,29,20


In [81]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [82]:
wnl = WordNetLemmatizer()
ps = PorterStemmer()

stopword = set(stopwords.words('english'))

In [83]:
def preprocessText(text):
    text = text.lower()

    tokens = word_tokenize(text)

    filtered = [token for token in tokens if token not in stopword and token.isalpha()]

    lemmas = [wnl.lemmatize(x) for x in filtered]

    return ' '.join(lemmas)


In [84]:
dataset['processedText'] = dataset['Text'].apply(preprocessText)
dataset['processedText']

0                            enjoying beautiful day park
1                               traffic terrible morning
2                               finished amazing workout
3                       excited upcoming weekend getaway
4                       trying new recipe dinner tonight
                             ...                        
727    collaborating science project received recogni...
728    attending surprise birthday party organized fr...
729    successfully fundraising school charity initia...
730    participating multicultural festival celebrati...
731    organizing virtual talent show challenging tim...
Name: processedText, Length: 732, dtype: object

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [86]:
X = dataset['processedText']
y = dataset['Sentiment']

In [87]:
y = y.str.strip().str.lower() 

In [88]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y_encoded = encoder.fit_transform(y)

In [89]:
print(np.unique(y_encoded))  

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190]


In [90]:
y_encoded

array([146, 134, 146, 146, 135, 146, 146, 146, 134, 135, 146, 134, 146,
       146, 135, 146, 146, 146, 135, 134, 146, 146, 146, 146, 146, 146,
       146, 146, 146, 146, 146, 146, 146, 146, 146, 146, 146, 146, 146,
       146, 146, 146, 146, 146, 146, 146, 146, 146, 146, 146, 146, 146,
        10,  79, 164,  56,  92, 116, 124,   9,  70,   2,   6,  17,  54,
       174,   0,   3,  11,  20,  26,  37,  76, 119, 149, 167,  10,  79,
       164,  56,  92, 116, 124,   9,  70,   2,   6,  17,  54, 174,   0,
         3,  11,  20,  26,  37,  76, 119, 149, 167,   0,  76,  20,  26,
        37,  76, 119, 149, 167,  60,  75,  40, 166,  90, 100,  66,  33,
       177,  15,  71,  87, 159,  60,  75,  40, 166,  90, 100,  66,  33,
       177,  15,  71,  87, 159,  60,  60,  75,  40, 166,  90, 100,  66,
        33, 177,  15,  71,  87, 159,  60,  75,  40, 166,  90, 100,  66,
        33, 177,  15,  71,  87, 159,  60,  50,  91, 121, 114, 157,  86,
        24,  12, 110,  99,  74, 153,  56,  50,  91, 121, 114, 15

In [91]:
list(encoder.inverse_transform([134]))

['negative']

In [92]:
vectorizer = TfidfVectorizer()

x_vect = vectorizer.fit_transform(X)

In [93]:
xtrain, xtest, ytrain, ytest = train_test_split(x_vect, y_encoded, test_size=0.2, random_state=42)

In [94]:
xtrain

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4659 stored elements and shape (585, 2234)>

In [95]:
ytrain

array([163, 150,  26,  46, 161,  60, 166, 146,  14,  94,  76, 101,  15,
       174,  71,  21,  18, 146, 167,  49,  92,  52,  37,  90, 108, 146,
       116,  21,  49,  50, 165,  75, 124, 146, 116, 165, 116, 116,  46,
        68,  92, 163, 135,  69,  40,  34,  76,   9,   2,  52, 173, 146,
         8,   0,  97,  95,  24, 152,  40, 116, 174,  75,  55, 116, 137,
       153,  17,  50, 189,  31, 180,  34,  75,   0,  18, 146,  50,  48,
       155, 135, 135, 111, 136,  93,  37, 100, 131, 126,  76, 101,  11,
       146, 174, 106,  83,  91, 116, 116,  52, 172, 128, 134,  90, 146,
       146, 146, 119, 157, 108,  90, 135, 121,  76, 116,  91,   7,  76,
       101,  50, 146, 112, 109,  86, 142,  59,   4,  90, 144, 151, 135,
       101, 146, 136,  76, 135,  90, 116,   0,  19,  20,  17, 187,  50,
        65, 110,  76,  57,  90, 167, 143, 101,  11, 116,  56,  82, 106,
        54, 116, 177, 166, 163, 144, 137, 146,  93,  71,  90,  76,  34,
        33, 137,  39,   1, 101,  50,  17, 177,  55,  91, 135, 14

In [96]:
model = MultinomialNB()

In [97]:
model.fit(xtrain, ytrain)

MultinomialNB()

In [98]:
ypred = model.predict(xtest)

In [99]:
accuracy_score(ytest, ypred)

0.1360544217687075

In [101]:
confusion_matrix(ytest, ypred)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(84, 84))